In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import pystan

gini = pd.read_csv('swiid8_3_summary.csv',usecols=['country','year','gini_disp'])
gdp = pd.read_stata('pwt91.dta',columns=['country','year','rgdpe','avh','emp','csh_i','csh_x','hc'])

df = pd.merge(gini,gdp)
df['prod'] = np.log(df['rgdpe']/(df['avh']*df['emp']))
df['g'] = df.groupby('country')['prod'].diff().shift(-1)
df['gini_disp']*=1/100
df = df[['country','year','g','prod','hc','csh_i','csh_x','gini_disp']].dropna()
df.head()

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


,country,year,g,prod,hc,csh_i,csh_x,gini_disp
73,Argentina,1961,-0.058088,1.315179,1.964928,0.156095,0.081157,0.361
74,Argentina,1962,-0.017734,1.257091,1.976053,0.136322,0.111727,0.359
75,Argentina,1963,0.057077,1.239357,1.987240,0.117448,0.106276,0.358
76,Argentina,1964,0.132784,1.296434,1.998491,0.145034,0.095864,0.358
77,Argentina,1965,-0.014582,1.429217,2.009806,0.133738,0.077476,0.357


In [15]:
stan_code = '''
    data {
        int<lower=1> K; // Number of countries
        int<lower=1> J; // Number of years
        int<lower=1> N; // Number of observations
        int<lower=1,upper=K> kk[N]; // Country for observation n
        int<lower=1,upper=J> jj[N]; // Year for observation n
        real g[N]; // Growth rate
        real y[N]; // Productivity level
        
        int<lower=0> C; // Number of covariates
        matrix[N, K] x; // Covariate matrix
        
    }
    parameters {
        vector[C] beta; // Coefficients for covariates
        
        real<lower=1> mu_phi;
        real<lower=0> sigma_phi;
        real<upper=0> lambda; // Convergence rate
        real<lower=0> gamma; // steady-state growth rate
        real<lower=0> sigma;
    }
    model {
        phi ~ normal(mu_phi,sigma_phi);
        g ~ normal(gamma + lambda*(y-phi) + x*beta,sigma);
    }
'''

stan_data = {
    'N': len(df),
    'K': 4,
    'x': df[['hc','csh_i','csh_x','gini_disp']],
    'g': df['g'],
    'y': df['prod']
}

sm = pystan.StanModel(model_code=stan_code)
print('Done!')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2054a492557c5e1347a8b8b4653931f0 NOW.


In [16]:
fit = sm.sampling(data=stan_data,iter=500,chains=2)
print(fit)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Inference for Stan model: anon_model_2054a492557c5e1347a8b8b4653931f0.
2 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=500.

             mean se_mean     sd    2.5%     25%     50%     75%   97.5%  n_eff   Rhat
phi[1]      -1.25    1.65   2.22   -5.89   -2.72   -1.06    0.56    2.01      2   1.51
phi[2]      -0.09    0.99   1.87   -3.82   -1.32 -1.5e-3    1.31    3.42      4   1.19
phi[3]       2.17    0.16   1.55   -0.83    1.13     2.1    3.21    5.39     96   1.03
phi[4]       4.28    1.59   2.32    0.34    2.56    4.08    5.82    9.48      2   1.46
phi[5]       0.24     0.7   1.59   -3.12   -0.74    0.33    1.34    2.96      5   1.15
phi[6]       1.49    0.07   1.51   -1.64     0.5    1.58     2.4    4.59    450    1.0
phi[7]        1.1    0.08   1.42   -1.91    0.23    1.16    2.03    3.84    328   1.02
phi[8]       2.71    0.82   1.58   -0.37    1.65    2.69     3.6    6.13      4    1.2
phi[9]       0.79    0.18   1.51